In [1]:
using DataStructures
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using BenchmarkTools

## Funzione SPHERE()

In [2]:
@enum surface triangled=1 single=2
function sphere(radius=1., angle1=pi, angle2=2*pi, surface=triangled)
    function sphere0(shape=[18, 36])
        V, CV = simplexGrid(shape)
        V = [angle1/shape[1] 0;0 angle2/shape[2]]*V
        V = broadcast(+, V, [-angle1/2, -angle2/2])
        W = [V[:, k] for k=1:size(V, 2)]
        V = hcat(map(p->let(u, v)=p;[radius*cos(u)*cos(v);
            radius*cos(u)*sin(v);radius*sin(u)]end, W)...)
        W, CW = simplifyCells(V, CV)
        CW = [triangle for triangle in CW if length(triangle)==3]
        if Int(surface)==1
            return W, CW
        elseif Int(surface)==2
            return W,[collect(1:size(W, 2))]
        end
    end
    return sphere0
end

sphere (generic function with 5 methods)

In [3]:
@btime W,CW = Lar.sphere()();

  3.686 ms (31870 allocations: 3.09 MiB)


In [4]:
@code_warntype Lar.sphere()();

Variables
  #self#::LinearAlgebraicRepresentation.var"#sphere0#296"{Float64,Irrational{:π},Float64,LinearAlgebraicRepresentation.surface}

Body::Union{Nothing, Tuple{Array,Array{Array{Int64,1},1}}}
1 ─ %1 = Base.vect(18, 36)::Array{Int64,1}
│   %2 = (#self#)(%1)::Union{Nothing, Tuple{Array,Array{Array{Int64,1},1}}}
└──      return %2


La funzione è type unstable poichè Body::Union{Nothing, Tuple{Array,Array{Array{Int64,1},1}}}. Il problema è che la funzione può ritornare una tupla di Int oppure nothing a seconda della condizione dell'if ...

In [3]:
@benchmark Lar.sphere()()

BenchmarkTools.Trial: 1229 samples with 1 evaluation.
 Range (min … max):  3.107 ms … 12.117 ms  ┊ GC (min … max): 0.00% … 56.19%
 Time  (median):     3.487 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.053 ms ±  1.326 ms  ┊ GC (mean ± σ):  6.79% ± 12.85%

  █▇▅▅▄▄▃▃▃▂▂▁▂▁ ▁▂ ▁▁           ▁▁                           
  █████████████████████▇█▆▆▅▇█▆▄███▆▁▆▅▄▄▁▄▄▅▁▅▁▅▄▅▁▅▄▅▅▅▁▅▄ █
  3.11 ms      Histogram: log(frequency) by time     9.65 ms <

 Memory estimate: 3.09 MiB, allocs estimate: 31868.

## Uso della macro @threads 

#### Per settare il numero di threads pari a 2 ho dovuto per forza eseguire (come scritto nella guida di julia) i seguenti passaggi :  export JULIA_NUM_THREADS=2  , set JULIA_NUM_THREADS=2 , il numero di threads va settato in base alla macchina detenuta , se non lo si fa l'uso dei threads è nullo .

In [5]:
using Base.Threads
Threads.nthreads() = 2  #setto il numero di threads pari a 2
nthreads()

2

In [9]:
function simplifyCellsRef(V,CV)
    PRECISION = 5
    vertDict = DefaultDict{Array{Float64,1}, Int64}(0)
    index = 0
    W = Array{Float64,1}[]
    FW = Array{Int64,1}[]
    
    for incell in CV
        outcell = Int64[]
        for v in incell
         vert = V[:,v]
         key = map(approxVal(PRECISION), vert)
            if vertDict[key]==0
                index += 1
                vertDict[key] = index
                push!(outcell, index)
                push!(W,key)
            else
                push!(outcell, vertDict[key])
            end
        end
         push!(FW, [Set(outcell)...])
    end
    return hcat(W...),FW
end

simplifyCellsRef (generic function with 1 method)

In [10]:
function approxVal(PRECISION)
    function approxVal0(value)
        out = round(value, digits=PRECISION)
        if out == -0.0
            out = 0.0
        end
        return out
    end
    return approxVal0
end

approxVal (generic function with 1 method)

In [6]:
@enum surface triangled=1 single=2
function sphere1(radius=1., angle1=pi, angle2=2*pi, surface=triangled)
    function sphere01(shape=[18, 36])
        V, CV = Lar.simplexGrid(shape)
        V = [angle1/shape[1] 0;0 angle2/shape[2]]*V
        V = broadcast(+, V, [-angle1/2, -angle2/2])
        n = size(V,2)
        W = Array{Array{Float64,1}}(undef, n)
        
        @inbounds @threads for k=1:n
         W[k] = V[:,k] 
    end
        V = hcat(map(p->let(u, v)=p;[radius*cos(u)*cos(v);
            radius*cos(u)*sin(v);radius*sin(u)]end, W)...)
        W, CW = simplifyCellsRef(V, CV)
        filter!(triangle -> length(triangle)==3,CW)
        if Int(surface)==1
            return W, CW
        elseif Int(surface)==2
            return W,[collect(1:size(W, 2))]
        end
    end
    return sphere01
end

sphere1 (generic function with 5 methods)

In [11]:
@btime W,CW = sphere1()();

  2.340 ms (23096 allocations: 2.26 MiB)


Prestazioni aumentate di poco . Ho usato le funzioni di supporto(refactoring) per poter poi modificare e interagire con la mia versione della funzione sphere , ho cercato di utilizzare i threads quindi per farlo ho dovuto "esternare" il ciclo for , cosi facendo ho potuto usare la macro @threads . Di fatti i cambiamenti non sono efficienti e spesso meno comprensibili e piu disordinati ma solo in questo modo sono state possibili applicare le macro . Senza l'uso delle macro , con il seguente ciclo for ho notato che la velocità di esecuzione aumenta di molto ( si puo notare nel notebook "disk" , nel caso di disk ho deciso di non omettere nessun passaggio e far vedere tutta l'evoluzione del codice stampandone i risultati legati alla velocità di esecuzione . ) In più è stata modificata la prima condizione sull'if , modificata con un filter. Questo messaggio è comune e sarà riportato alla fine di ogni notebook .